In [1]:
import os
os.chdir("../")

In [2]:
import pandas as pd
df = pd.read_csv("data/WIN_H1.csv")

In [3]:
from functools import reduce
from backtest import Strategy, Backtest


import talib


class SmaCross(Strategy):

    def __init__(self):

        self.ma1_window = 12
        self.ma2_window = 50
        self.column_ma1 = 'Close'
        self.column_ma2 = 'Close'
        self.stop_loss = -100
        self.stop_gain = 200
    def create_indicators(self, dataframe):
        
        dataframe['SMA_short'] = talib.SMA(dataframe[self.column_ma1],
                                           self.ma1_window)
        dataframe['SMA_long'] = talib.SMA(dataframe[self.column_ma2],
                                          self.ma2_window)
        
        return dataframe
    def buy_entry(self, dataframe):
        conditions = []
        conditions.append(dataframe['SMA_short'] > dataframe['SMA_long'])
        conditions.append(dataframe['SMA_short'].shift(1) <= dataframe['SMA_long'].shift(1))
        conditions.append(dataframe['Volume'] > 0)
        dataframe.loc[reduce(lambda x, y: x & y, conditions), 'buy_entry'] = 1
        return dataframe

    def buy_exit(self, dataframe):
        conditions = []
        conditions.append(dataframe['SMA_long'] > dataframe['SMA_short'])
        conditions.append(dataframe['SMA_long'].shift(1) <= dataframe['SMA_short'].shift(1))
        conditions.append(dataframe['Volume'] > 0)
        dataframe.loc[reduce(lambda x, y: x & y, conditions), 'buy_exit'] = 1
        return dataframe
    def sell_entry(self, dataframe):
        conditions = []
        conditions.append(dataframe['SMA_long'] > dataframe['SMA_short'])
        conditions.append(dataframe['SMA_long'].shift(1) <= dataframe['SMA_short'].shift(1))
        conditions.append(dataframe['Volume'] > 0)
        dataframe.loc[reduce(lambda x, y: x & y, conditions), 'sell_entry'] = 1
        return dataframe
    def sell_exit(self, dataframe):
        conditions = []
        conditions.append(dataframe['SMA_short'] > dataframe['SMA_long'])
        conditions.append(dataframe['SMA_short'].shift(1) <= dataframe['SMA_long'].shift(1))
        conditions.append(dataframe['Volume'] > 0)
        dataframe.loc[reduce(lambda x, y: x & y, conditions), 'sell_exit'] = 1
        return dataframe

In [4]:
strategy = SmaCross()

backtest = Backtest(df, strategy)
result = backtest.run_backtest()

In [5]:
result

{'all_trades_gain': [-43.800000000000004,
  84.2,
  -53.400000000000006,
  169.60000000000002,
  -341.40000000000003,
  -208.0,
  -114.2,
  313.8,
  -179.8,
  -236.8,
  -118.4,
  -39.0,
  -26.400000000000002,
  -340.40000000000003,
  -79.2,
  249.60000000000002,
  -200.8,
  -109.80000000000001,
  104.60000000000001,
  -114.0,
  -118.0,
  212.20000000000002,
  -141.20000000000002,
  -110.2,
  -49.400000000000006,
  -94.60000000000001,
  -507.8,
  -8.200000000000001,
  -190.60000000000002,
  101.0,
  -138.0,
  -29.6,
  -25.400000000000002,
  -45.800000000000004,
  113.0,
  -33.4,
  59.2,
  -119.0,
  -226.0],
 'total_gain': -2635.4000000000005,
 'mean_gain': -67.57435897435897,
 'trades_info': [{'type': 'buy',
   'init': 136816.0,
   'end': 136597.0,
   'date': '2021.07.12 16:00'},
  {'type': 'sell',
   'init': 135365.0,
   'end': 134944.0,
   'date': '2021.07.16 15:00'},
  {'type': 'buy',
   'init': 135296.0,
   'end': 135029.0,
   'date': '2021.07.22 15:00'},
  {'type': 'buy',
   'init'